In [1]:
import os
from PIL import Image
from tqdm import tqdm
from skimage import color
from skimage import io
from skimage import img_as_ubyte
from skimage.transform import resize
import numpy as np
import h5py

In [6]:
image_folder = 'data/data/'
SIZE = 128
taget_size = (SIZE,SIZE)
save_folder = 'images/bedrooms/'

In [4]:
print(image_folder)
files = os.listdir(image_folder)
x = 0
y = 0
for file in tqdm(files):
    if file[-4:] != 'webp':
        continue
    image = np.array(Image.open(image_folder+file))
    s = image.shape
    x+=s[0]
    y+=s[1]
print(x/len(files))
print(y/len(files))

data/data/


100%|██████████████████████████████████████████████████████████████████| 6000/6000 [00:28<00:00, 210.89it/s]


264.81033333333335
356.45683333333335


In [7]:
# resize
take = 6000
files = os.listdir(image_folder)
for file in tqdm(files[:take]):
    image = Image.open(image_folder+file)
    image = image.resize(taget_size, Image.BICUBIC)
    file = file.replace('webp','jpg')
    image.save(save_folder+file,"jpeg")

100%|██████████████████████████████████████████████████████████████████| 6000/6000 [00:37<00:00, 160.80it/s]


In [8]:
# build dataset
files = list(os.listdir(save_folder))
count = len(files) 
i = 0 
gray = True
l = taget_size[0]*taget_size[1]*(1 if gray else 3)
with h5py.File('data.hdf5','w') as f: 
    dset = f.create_dataset('ints', (count,l), dtype='uint8')
    i = 0
    for file in tqdm(files):
        if gray:
            conv = 'L'
        else:
            conv='RGB'
        image = Image.open(save_folder+file).convert(conv)
        image = np.array(image)
        image = image.reshape(l).astype(int) 
        dset[i] = image
        i+=1
        
    

100%|██████████████████████████████████████████████████████████████████| 6000/6000 [00:19<00:00, 314.56it/s]


In [19]:
with h5py.File('data.hdf5','r') as f: 
    a = f['ints']
    imfile = Image.fromarray(a[0].reshape(taget_size[0],taget_size[1]))
    imfile.save("result_bw.png")

In [7]:
imfile = Image.fromarray(dset[0])
imfile.save("result_bw.png")

ValueError: Not a dataset (not a dataset)

In [32]:
64*64*3

12288